In [4]:
import tarfile
import gzip
import os

## Decompress the raw data files

DATA_DIR = r'data'
DATA_SOURCE = r'GSE166992_RAW.tar'

with tarfile.open(DATA_SOURCE, 'r') as tar:
    tar.extractall(path=DATA_DIR)


for file in os.listdir(DATA_DIR):

    path = os.path.join(DATA_DIR, file)
    out_path = path[:-3]  # remove '.gz'

    with gzip.open(path, 'rb') as read:
        with open(out_path, 'wb') as f_out:
            f_out.writelines(read)

    os.remove(path)


In [5]:
# load the data into anndata object
import re
import scanpy as sc
from scipy.io import mmread
import anndata as ad
import pandas as pd

HEALTHY_SAMPLES = ['GSM5090454', 'GSM5090448', 'GSM5090446']

samples = {}

for file in os.listdir(DATA_DIR):
    match = re.match(r"(GSM\d+_[^_]+_5DGE)_(barcodes|features|matrix)\.(tsv|mtx)", file)
    sample_id = match.group(1)
    file_type = match.group(2)
    samples.setdefault(sample_id, {})[file_type] = os.path.join(DATA_DIR, file)

adatas = []

for sample, data_paths in samples.items():

    matrix = mmread(data_paths["matrix"]).tocsr().T

    features = pd.read_csv(data_paths["features"], sep="\t", header=None).astype(str)
    features.columns = ['gene_id', 'gene_name', 'feature_type']
    features.index = features["gene_id"].astype(str)

    adata = ad.AnnData(
        X=matrix,
        var = features
    )

    sample_id = sample[:10]
    adata.obs["sample_id"] = sample_id
    
    if sample_id in HEALTHY_SAMPLES:
        adata.obs["disease_status"] = "healthy"
    else:
        adata.obs["disease_status"] = "covid_19"

    adatas.append(adata)

adata = ad.concat(adatas, join='outer', axis=0)
adata.var = adatas[0].var

c:\Users\jpdyc\Coding Files\BIOF501 Project\venv\Lib\site-packages\anndata\_core\anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
import celltypist
import scanpy as sc

# First, save the raw adata for future reference
adata.raw = adata

# Next, normalize and log-transform the data as required by celltypist
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Celltypist requires var_names to be gene names
adata.var_names = adata.var['gene_name'].values

pred = celltypist.annotate(adata, model="Adult_COVID19_PBMC.pkl")
adata.obs["cell_type"] = pred.predicted_labels

c:\Users\jpdyc\Coding Files\BIOF501 Project\venv\Lib\site-packages\celltypist\classifier.py:11: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('scanpy')` instead
  from scanpy import __version__ as scv
🔬 Input data has 63895 cells and 33538 genes
🔗 Matching reference genes in the model
🧬 2989 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!


In [7]:
print(adata.obs['cell_type'].value_counts())

cell_type
CD14 Monocyte       13709
RBC                 11902
CD4n T              10782
CD8m T               6865
CD4m T               5156
B                    4866
NK                   4219
Platelet             2093
CD16 Monocyte        1796
gd T                 1163
DC                    320
CD4 T                 265
CD8eff T              216
pDC                   180
IgA PB                140
SC & Eosinophil       112
IgG PB                 56
Class-switched B       49
Neutrophil              6
Name: count, dtype: int64


In [9]:
import scvi

# scVI expects raw counts 
# TODO just train scVI before celltypist to avoid this step
adata.X = adata.raw.X.copy()

scvi.model.SCVI.setup_anndata(adata)

model = scvi.model.SCVI(adata)
model.train()
model.save("scvi_model_covid19_pbmc", overwrite=True)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Epoch 2/125:   1%|          | 1/125 [04:14<8:46:35, 254.80s/it, v_num=1, train_loss=4.74e+3]


Detected KeyboardInterrupt, attempting graceful shutdown ...


Exception raised during training. <class 'NameError'> 1


SystemExit: 1